In [1]:
import os
os.chdir("/home/mayank_khulbe_farmart_co/fmt/document-alignment-detection")

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

from PIL import Image

from document_alignment_correction.helper import rotateImage
from skimage import io

In [3]:
os.chdir("/home/mayank_khulbe_farmart_co/fmt/document-alignment-detection/experiments/mayank")

In [4]:
image = io.imread("data/ffbf0023_4.png")
image

array([[255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255],
       ...,
       [255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255]], dtype=uint8)

In [22]:

rotated_cropped2, rotated_2 = rotateImage(image, 123)
rotated_cropped2

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [23]:
np.unique(rotated_cropped2)

array([0.00000000e+00, 4.66715692e-06, 4.75792502e-06, ...,
       9.99999164e-01, 9.99999824e-01, 1.00000000e+00])

In [24]:
rotated_cropped3, rotated_3 = rotateImage(rotated_cropped2, -123)
rotated_cropped3

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.06689872, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.59994438, ..., 0.56772739, 0.02437766,
        0.        ],
       ...,
       [0.02145318, 0.29174507, 0.88623828, ..., 0.69942039, 0.01810148,
        0.        ],
       [0.        , 0.        , 0.31743955, ..., 0.18775844, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [14]:
rotated_cropped4, rotated_4 = rotateImage(rotated_cropped, 90)
rotated_4

array([[  0,   6,   3, ...,   5,   0,   0],
       [  0, 145, 157, ..., 178,  48,   0],
       [ 31, 172, 255, ..., 231,  57,   3],
       ...,
       [ 17, 153, 233, ..., 226,  81,   0],
       [  0,   0,  87, ...,  75,   0,   0],
       [  0,   0,   0, ...,   5,   0,   0]], dtype=uint8)

In [15]:
_, rotated_5 = rotateImage(rotated_4, -90)
rotated_5

array([[  0,   0,  17, ...,  31,   0,   0],
       [  0,   0, 153, ..., 172, 145,   6],
       [  0,  87, 233, ..., 255, 157,   3],
       ...,
       [  5,  75, 226, ..., 231, 178,   5],
       [  0,   0,  81, ...,  57,  48,   0],
       [  0,   0,   0, ...,   3,   0,   0]], dtype=uint8)

In [16]:
io.imsave('data/output_image2.png', rotated_5)

In [ ]:
import numpy as np
from PIL import Image
import math

def remove_black_border(image):
    # Convert image to numpy array for easier manipulation
    image_array = np.array(image)

    # Find the bounding box of the image
    nonzero_pixels = np.argwhere(image_array > 0)
    min_row, min_col = np.min(nonzero_pixels, axis=0)
    max_row, max_col = np.max(nonzero_pixels, axis=0)

    # Crop the image to remove black borders
    cropped_image = image_array[min_row:max_row+1, min_col:max_col+1]

    # Convert the cropped image back to PIL Image format
    cropped_image = Image.fromarray(cropped_image)

    return cropped_image


def rotate_grayscale_image(image_path, degrees):
    # Open the image
    image = Image.open(image_path).convert('L')

    # Remove black border if present
    image = remove_black_border(image)

    # Convert degrees to radians
    radians = math.radians(degrees)

    # Calculate the size of the rotated image
    width, height = image.size
    new_width = int(math.ceil(abs(width * math.cos(radians)) + abs(height * math.sin(radians))))
    new_height = int(math.ceil(abs(width * math.sin(radians)) + abs(height * math.cos(radians))))

    # Calculate the center point of the original image
    center_x = width / 2
    center_y = height / 2

    # Create a new blank image with the calculated size
    rotated_image = Image.new('L', (new_width, new_height))

    # Iterate over each pixel in the rotated image
    for x in range(new_width):
        for y in range(new_height):
            # Calculate the corresponding pixel position in the original image
            src_x = int((x - new_width / 2) * math.cos(-radians) + (y - new_height / 2) * math.sin(-radians) + center_x)
            src_y = int(-(x - new_width / 2) * math.sin(-radians) + (y - new_height / 2) * math.cos(-radians) + center_y)

            # Check if the calculated position is within the original image boundaries
            if src_x >= 0 and src_x < width and src_y >= 0 and src_y < height:
                # Get the pixel value from the original image using nearest-neighbor interpolation
                pixel_value = image.getpixel((src_x, src_y))

                # Set the pixel value in the rotated image
                rotated_image.putpixel((x, y), pixel_value)

    return rotated_image


# # Usage example
# input_image_path = 'path/to/your/image.jpg'
# output_image_path = 'path/to/save/result.jpg'

# # Rotate the image to 123 degrees
# rotated_image = rotate_grayscale_image(input_image_path, 123)

# # Rotate the rotated image back to 0 degrees
# restored_image = rotate_grayscale_image(rotated_image, -123)

# # Save the restored image
# restored_image.save(output_image_path)


In [ ]:
import numpy as np
from PIL import Image
import math

def remove_black_border(image):
    # Convert image to numpy array for easier manipulation
    image_array = np.array(image)

    # Find the bounding box of the image
    nonzero_pixels = np.argwhere(image_array > 0)
    min_row, min_col = np.min(nonzero_pixels, axis=0)
    max_row, max_col = np.max(nonzero_pixels, axis=0)

    # Crop the image to remove black borders
    cropped_image = image_array[min_row:max_row+1, min_col:max_col+1]

    # Convert the cropped image back to PIL Image format
    cropped_image = Image.fromarray(cropped_image)

    return cropped_image


def rotate_grayscale_image(image_path, degrees):
    # Open the image
    image = Image.open(image_path).convert('L')

    # Remove black border if present
    image = remove_black_border(image)

    # Convert degrees to radians
    radians = math.radians(degrees)

    # Calculate the size of the rotated image
    width, height = image.size
    new_width = int(math.ceil(abs(width * math.cos(radians)) + abs(height * math.sin(radians))))
    new_height = int(math.ceil(abs(width * math.sin(radians)) + abs(height * math.cos(radians))))

    # Calculate the center point of the original image
    center_x = width / 2
    center_y = height / 2

    # Create a new blank image with the calculated size
    rotated_image = Image.new('L', (new_width, new_height))

    # Iterate over each pixel in the rotated image
    for x in range(new_width):
        for y in range(new_height):
            # Calculate the corresponding pixel position in the original image
            src_x = int((x - new_width / 2) * math.cos(-radians) + (y - new_height / 2) * math.sin(-radians) + center_x)
            src_y = int(-(x - new_width / 2) * math.sin(-radians) + (y - new_height / 2) * math.cos(-radians) + center_y)

            # Check if the calculated position is within the original image boundaries
            if src_x >= 0 and src_x < width and src_y >= 0 and src_y < height:
                # Get the pixel value from the original image using nearest-neighbor interpolation
                pixel_value = image.getpixel((src_x, src_y))

                # Set the pixel value in the rotated image
                rotated_image.putpixel((x, y), pixel_value)

    # Crop the rotated image to remove black borders
    rotated_image = remove_black_border(rotated_image)

    return rotated_image


In [ ]:
nn_rotated = rotate_grayscale_image("data/ffbf0023_4.png", 123)
np.array(nn_rotated).shape

In [ ]:
nn_rotated.save("data/nn_rotated.png")

In [ ]:
np.unique(np.array(nn_rotated))

In [ ]:
np.unique(np.array(image))

In [ ]:
orig_rotated = rotate_grayscale_image("data/nn_rotated.png", -123)
orig_rotated

In [ ]:
orig_rotated.save("data/orig_rotated.png")

In [ ]:
np.unique(np.array(orig_rotated))

In [ ]:
np.unique(np.array(image))

In [ ]:
np.unique(np.array(nn_rotated))

In [ ]:
np.array(image).shape

In [ ]:
np.array(orig_rotated)

In [ ]:
rotat2 = orig_rotated.rotate(90, expand=True)
rotat2

In [ ]:
rotat2.save("data/rotate_2.png")

In [ ]:
import torch
from document_alignment_correction.helper import rotateImage
from skimage import io

gray_image = io.imread("data/rotate_2.png")
_, fixed_image = rotateImage(gray_image, -90)
fixed_image

In [ ]:
pip install tqdm